# Sentiment analysis

## AI for research, 2024

The ultimate goal of this exercise consists performing the same exercise, namely sentiment analysis, 
using traditional NLP and GPT-4.

## The Dataset

We use the [News Sentiment Dataset](https://www.kaggle.com/datasets/yasserh/twitter-tweets-sentiment-dataset) from Kaggle.

In [2]:
cd tutorials/session_2

/files/tutorials/session_2


In [3]:
pwd

'/files/tutorials/session_2'

1. __Import Dataset as a pandas dataframe__

In [4]:
import pandas
df = pandas.read_csv("Tweets.csv")

In [5]:
df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will 🦈 miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [12]:
df.iloc[1200,:]['text']

' hahahah of course  they have such a nasty display picture :`)'

2. __Describe Dataset (text and graphs)__

3. __Split Dataset into training, validation and test set. What is the purpose of the validation set?__

## Text Mining

1. __Extract features from the training dataset. What do you do with non-words / punctuation?__

2. __Convert occurrencies to frequencies. Make another version with tf-idf.__

3. __Choose a classifier to predict the sentiment on the *validation* set. Compute the confusion matrix.__

## Sentiment Analysis using GPT completion

1. __Setup an openai key. Explore openai *completion* API.__

In [6]:
import openai

In [ ]:
# R

In [ ]:
# make sure we have the right version. 
openai.version.VERSION

'1.3.5'

In [ ]:
api_key = ""

In [30]:
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
)

response = client.completions.create(
    prompt = "Say this is a test",
    model="gpt-3.5-turbo-instruct",
)

In [35]:
print( response.choices[0].text )

 sentence

This is a test sentence.


In [21]:
# lookup on google "openai completion python"
# from the github website, copy the example text

In [22]:
tweet = df['text'][1200]
tweet

' hahahah of course  they have such a nasty display picture :`)'

In [36]:
prompt = "Classify the following text as, positive, negative or neutral:\n\n"
prompt += "Tweet: "
prompt += tweet
prompt += "\n\n"
prompt += "Answer: "


In [37]:
print(prompt)

Classify the following text as, positive, negative or neutral:

Tweet:  hahahah of course  they have such a nasty display picture :`)

Answer: 


In [40]:
model = "gpt-3.5-turbo-instruct"
response = client.completions.create(model=model, prompt=prompt, max_tokens=50)


generated_text = response.choices[0].text
print(generated_text)


Negative


In [41]:
# use string interpolation:
prompt = f"""Classify the following text as, positive, negative or neutral:

Tweet: {tweet}
Answer:
"""
print(prompt)

Classify the following text as, positive, negative or neutral:

Tweet:  hahahah of course  they have such a nasty display picture :`)
Answer:



2. __Design a prompt to extract the sentiment from a tweet. Test it on very few tweets from the training dataset. Propose different versions.__

3. __Write a function which takes in: the prompt template, the tweet text and returns the sentiment as an integer.__

## Performance shootout

1. __Compare the various methods on the test set.__